In [1]:
import numpy as np

from tqdm import tqdm

In [2]:
# allBoards is the powerset for [1,...,maxSum]
# https://leetcode.com/problems/subsets/description/
def getAllBoards(maxSum):
    nums = range(1, maxSum+1)
    solutions = []
    sub = []

    def dfs(i):
        if i == len(nums):
            solutions.append(frozenset(sub[:])) # need frozensets as dict keys
            return

        sub.append(nums[i])
        dfs(i+1) # with nums[i]
        sub.pop()
        dfs(i+1) # without nums[i]

    dfs(0)
    return solutions

# legalMoves are subsets of candidates which sum to target
# https://leetcode.com/problems/combination-sum-ii/description/
def getLegalMoves(candidates, target):
    solutions = []
    candidates.sort()

    def dfs(i, sol, remaining):
        if remaining == 0:
            solutions.append(set(sol[:]))
            return

        if remaining < 0 or i == len(candidates):
            return

        sol.append(candidates[i])
        dfs(i+1, sol, remaining - candidates[i])
        sol.pop()

        while i+1 < len(candidates) and candidates[i+1] == candidates[i]:
            i += 1

        dfs(i+1, sol, remaining)

    dfs(0, [], target)
    return solutions

# to play, specify {board : {diceSum : move}} for every board and diceSum
# score is sum of remaining tiles when moves run out
def play(moves):
    board = frozenset(range(1,maxSum+1))

    while len(board) > 0:
        d1 = np.random.randint(1, maxFace+1)
        d2 = np.random.randint(1, maxFace+1)
        move = moves[board][d1 + d2]

        if move is None:
            return sum(board)

        board = board - move

    return 0

In [3]:
maxSum = 12
maxFace = 6
allBoards = getAllBoards(maxSum)

In [4]:
# expected scores for playing optimally at every board
scores = {board : 0 for board in allBoards}

# bestMoves in response to every board and diceSum
bestMoves = {board : {diceSum : None for diceSum in range(2, maxSum+1)} for board in allBoards}

for board in sorted(allBoards, key=len): # grow tables from most solved to least solved board
    for d1 in range(1, maxFace+1):
        for d2 in range(1, maxFace+1):
            legalMoves = getLegalMoves(list(board), d1+d2)

            bestMove = None
            bestScore = sum(board)

            for legalMove in legalMoves:
                boardAfter = board - legalMove

                if scores[boardAfter] < bestScore:
                    bestMove = legalMove
                    bestScore = scores[boardAfter]

            bestMoves[board][d1+d2] = bestMove
            scores[board] += bestScore / (maxFace ** 2)

In [5]:
N = 100000
score = 0

for _ in tqdm(range(N)):
    score += play(bestMoves)

print(f"{(score / N):.6f}")

100%|██████████| 100000/100000 [00:06<00:00, 15181.20it/s]

35.091920
